<p style="background-color: #0055A4; font-size: 48px; font-weight: bold; color: white; padding: 20px; text-align: center;">
🧹 Data Cleaning 
</p>


In [10]:
#Import librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math 
from scipy.stats import pearsonr
import scipy.stats as stats
from plotly.subplots import make_subplots
from scipy.stats import shapiro
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore") 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
import plotly.graph_objects as go
import plotly.express as px
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import skew, shapiro
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
import itertools
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
import shap
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from collections import Counter
from sklearn.feature_selection import SelectKBest
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from prettytable import PrettyTable
import featuretools as ft

In [11]:
#Import data
train_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/fraudTrain.csv', delimiter=',', encoding='utf-8',index_col=0)
test_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/fraudTest.csv', delimiter=',', encoding='utf-8',index_col=0)



## 🔄 Check for duplicates

In [12]:
#train_df
duplicate_count = train_df.duplicated().sum()
print(f'Number of duplicate records: {duplicate_count}')


Number of duplicate records: 0


In [13]:
#train_df
duplicate_count = test_df.duplicated().sum()
print(f'Number of duplicate records: {duplicate_count}')


Number of duplicate records: 0


## ✨ Data Transformation

## Log Transformation

In [14]:
#train_df + test_df
skewed_cols = ['amt','city_pop']
for col in skewed_cols:
    train_df[col] = np.log1p(train_df[col])  
    test_df[col] = np.log1p(test_df[col])
     




In [15]:
results = []
for var in  skewed_cols:
    skewness = stats.skew(train_df[var])
    kurtosis = stats.kurtosis(train_df[var])
    
    results.append({
        'Feature':  var ,
        'Skewness': skewness,
        'Kurtosis': kurtosis
    })

results_df = pd.DataFrame(results)

display(results_df)

,Feature,Skewness,Kurtosis
0,amt,-0.298852,-0.527247
1,city_pop,0.606094,-0.301258


## Handle outliers by capping them 

In [16]:
numerical_vars_filtered = [
    "amt",
    "zip",
    "lat",
    "long",
    "city_pop",
    "unix_time",
    "merch_lat",
    "merch_long"
]


In [17]:
#train_df
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.348234e-01  7.532865e+00             818       1296675
1         zip -4.247050e+04  1.407495e+05               0       1296675
2         lat  2.364065e+01  5.292025e+01            4679       1296675
3        long -1.217580e+02 -5.519800e+01           49922       1296675
4    city_pop  1.650397e+00  1.488145e+01            4168       1296675
5   unix_time  1.307799e+09  1.390337e+09               0       1296675
6   merch_lat  2.389818e+01  5.279255e+01            4967       1296675
7  merch_long -1.218880e+02 -5.524608e+01           41994       1296675


In [19]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    return df

columns_with_outliers = [
    "amt",
    "lat",
    "long",
    "city_pop",
    "merch_lat",
    "merch_long"
]

for col in columns_with_outliers:
    train_df = handle_outliers(train_df, col)


In [20]:
# results after capping outliers
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = train_df[(train_df[col] < lower_bound) | (train_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.348234e-01  7.532865e+00               0       1296675
1         zip -4.247050e+04  1.407495e+05               0       1296675
2         lat  2.364065e+01  5.292025e+01               0       1296675
3        long -1.217580e+02 -5.519800e+01               0       1296675
4    city_pop  1.650397e+00  1.488145e+01               0       1296675
5   unix_time  1.307799e+09  1.390337e+09               0       1296675
6   merch_lat  2.389818e+01  5.279255e+01               0       1296675
7  merch_long -1.218880e+02 -5.524608e+01               0       1296675


In [21]:
#test_df
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = test_df[col].quantile(0.25)
    Q3 = test_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = test_df[(test_df[col] < lower_bound) | (test_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.372033e-01  7.531819e+00             402       1296675
1         zip -4.228650e+04  1.405895e+05               0       1296675
2         lat  2.383005e+01  5.273365e+01            1933       1296675
3        long -1.217322e+02 -5.524100e+01           21104       1296675
4    city_pop  1.691879e+00  1.480513e+01            1697       1296675
5   unix_time  1.361271e+09  1.400625e+09               0       1296675
6   merch_lat  2.395701e+01  5.275246e+01            2090       1296675
7  merch_long -1.218659e+02 -5.530390e+01           17926       1296675


In [22]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    return df

columns_with_outliers = [
    "amt",
    "lat",
    "long",
    "city_pop",
    "merch_lat",
    "merch_long"
]

for col in columns_with_outliers:
    train_df = handle_outliers(test_df, col)


In [23]:
# results after capping outliers
outliers_info = []

for col in numerical_vars_filtered:
    Q1 = test_df[col].quantile(0.25)
    Q3 = test_df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = test_df[(test_df[col] < lower_bound) | (test_df[col] > upper_bound)]
    
    outliers_info.append({
        'Feature': col,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Total Points': len(train_df)
    })

outliers_df = pd.DataFrame(outliers_info)
print(outliers_df)


      Feature   Lower Bound   Upper Bound  Outliers Count  Total Points
0         amt -7.372033e-01  7.531819e+00               0        555719
1         zip -4.228650e+04  1.405895e+05               0        555719
2         lat  2.383005e+01  5.273365e+01               0        555719
3        long -1.217322e+02 -5.524100e+01               0        555719
4    city_pop  1.691879e+00  1.480513e+01               0        555719
5   unix_time  1.361271e+09  1.400625e+09               0        555719
6   merch_lat  2.395701e+01  5.275246e+01               0        555719
7  merch_long -1.218659e+02 -5.530390e+01               0        555719
